In [4]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")
data=pd.read_sql_query("Select distinct date from allmatches --where date > '2023-04-24'",con=devengine)
#data=data.drop(columns='index')
data=data[(data['Date']>'2023-04-24')]
#data=data.sort_values(by=['Date','Tournament'])
#data.to_sql('AllMatches',index=False, con=devengine,if_exists='replace')
bankroll=1000
data
pd.read_sql_query("Select distinct * from allmatches where date = '2023-06-03'",con=devengine)

,Date,Sex,Tournament,Player_1,Player_2,Player_1_Odds,Player_2_Odds,Surface,Player_1_Rank,Player_2_Rank


In [180]:
def calculate_percentage_increase(initial_value, final_value):
    percentage_increase = ((final_value - initial_value) / initial_value) * 100
    return "{:.1f}%".format(percentage_increase)

print(calculate_percentage_increase(50,53))

6.0%


In [181]:
push_to_sql=False
Date='2023-05-25'

In [182]:
#Hard
STAKE=bankroll*0.01
hard=pd.read_sql_query(
    f"Select DISTINCT * From results_hard_1",con=devengine,)
# calculate Kelly stake for each row
hard['Dog_Odds']=hard['Dog_Odds'].astype(float)
hard['Fav_Odds']=hard['Fav_Odds'].astype(float)
hard_filtered=hard.copy()
hard_filtered['Win']=hard_filtered.apply(lambda x: True if x['Winner']==x['Dog'] else False, axis=1)
hard_filtered['Profit']=hard_filtered.apply(lambda x: -STAKE if x['Win']==False else (x['Dog_Odds']*STAKE)-STAKE,axis=1)
roi=calculate_percentage_increase(bankroll,bankroll+hard_filtered['Profit'].sum())
hard_filtered=hard_filtered[(hard_filtered['Dog_Odds']).ge(2)&(hard_filtered['Dog_Odds']).le(2.2)]
#print(len(hard_filtered[hard_filtered['Winner']==hard_filtered['Dog']])/len(hard_filtered))

hard_filtered['Fav_Elo']=hard_filtered.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']==x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
hard_filtered['Dog_Elo']=hard_filtered.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']!=x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
hard_filtered['rank_diff']=hard_filtered['dog_rank']-hard_filtered['fav_rank']
hard_filtered['elo_diff']=hard_filtered['Fav_Elo']-hard_filtered['Dog_Elo']
merge=hard_filtered[['Win','fav_percent','dog_percent','rank_diff','elo_diff']]

hard_today=pd.read_pickle('.\Hard_Today')
if hard_today.empty==False:
    # calculate Kelly stake for each row
    hard_today['Dog_Odds']=hard_today['Dog_Odds'].astype(float)
    hard_today['Fav_Odds']=hard_today['Fav_Odds'].astype(float)
    hard_today['Fav_Elo']=hard_today.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']==x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
    hard_today['Dog_Elo']=hard_today.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']!=x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
    hard_today['rank_diff']=hard_today['dog_rank']-hard_today['fav_rank']
    hard_today['elo_diff']=hard_today['Fav_Elo']-hard_today['Dog_Elo']
    hard_filtered=hard_today
    hard_filtered=hard_filtered[(hard_filtered['Dog_Odds']).ge(2)&(hard_filtered['Dog_Odds']).le(2.2)]
    hard_input=hard_filtered[['fav_percent','dog_percent','rank_diff','elo_diff']]


    if len(hard_input)>0:
        from sklearn.linear_model import LogisticRegression
        from sklearn.model_selection import train_test_split
        import numpy as np

        # Initialize an empty list to store probabilities from each iteration
        all_probabilities = pd.DataFrame()
        counter=0
        for q in range(1, 1000):
            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(
                merge[['fav_percent','dog_percent','rank_diff','elo_diff']],
                merge['Win'],
                test_size=0.3,
            )

            # Create a logistic regression model and fit it to the training data
            logreg = LogisticRegression(max_iter=1000000000000000000000)
            logreg.fit(X_train, y_train)

            # Evaluate the model on the testing data
            score = logreg.score(X_test, y_test)

            if score > 0.5:
                counter=counter+1
                hard_input = hard_filtered[['fav_percent','dog_percent','rank_diff','elo_diff']].copy()

                # Predict the probability of each sample belonging to class 1
                prob = logreg.predict_proba(hard_input)[:, 1]

                # Add 'Probability' column to the input DataFrame
                hard_input['Probability'] = prob

                xo = hard_input.merge(hard_filtered, left_index=True, right_index=True, suffixes=['','_y'])[['Fav','Dog','Dog_Odds','Probability','Time','Fav_Odds']]
                #print(xo)
                #print(all_probabilities)
                all_probabilities=pd.concat([all_probabilities,xo])
        if len(all_probabilities)>0:
            avg_probs = all_probabilities.groupby(['Time','Dog_Odds','Dog','Fav','Fav_Odds'])['Probability'].mean().reset_index()
            print(counter)
            avg_probs['Prediction']=avg_probs.apply(lambda x: True if x['Probability']>=0.5 else False,axis=1 )
            avg_probs['Date']=Date
            avg_probs['Surface']='Hard'
            avg_probs['Result']=None
            if push_to_sql:
                avg_probs[['Date','Dog_Odds','Dog','Fav','Prediction','Probability','Surface','Fav_Odds']].to_sql('Predictions_Today',con=devengine,index=False,if_exists='append')

            today_hard=(avg_probs[['Time','Dog_Odds','Dog','Fav','Prediction','Probability','Fav_Odds']])
    else:
        today_hard=pd.DataFrame()
        print('x')
else:
        today_hard=pd.DataFrame()
        print('x')

x


In [183]:
today_hard

""


In [184]:
#clay
STAKE=bankroll*0.01
clay=pd.read_sql_query(
    f"Select DISTINCT * From results_clay_1",con=devengine,)
# calculate Kelly stake for each row
clay['Dog_Odds']=clay['Dog_Odds'].astype(float)
clay['Fav_Odds']=clay['Fav_Odds'].astype(float)
clay_filtered=clay
clay_filtered['Win']=clay_filtered.apply(lambda x: True if x['Winner']==x['Dog'] else False, axis=1)
clay_filtered['Profit']=clay_filtered.apply(lambda x: -STAKE if x['Win']==False else (x['Dog_Odds']*STAKE)-STAKE,axis=1)
roi=calculate_percentage_increase(bankroll,bankroll+clay_filtered['Profit'].sum())

clay_filtered['Fav_Elo']=clay_filtered.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']==x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
clay_filtered['Dog_Elo']=clay_filtered.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']!=x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
clay_filtered['rank_diff']=clay_filtered['dog_rank']-clay_filtered['fav_rank']
clay_filtered['elo_diff']=clay_filtered['Fav_Elo']-clay_filtered['Dog_Elo']
clay_filtered=clay_filtered[(clay_filtered['Dog_Odds']).ge(2)&(clay_filtered['Dog_Odds']).le(2.2)]
merge=clay_filtered[['Win','fav_percent','dog_percent','rank_diff','elo_diff']]


clay_today=pd.read_pickle(r'.\Clay_Today')
# calculate Kelly stake for each row
clay_today['Dog_Odds']=clay_today['Dog_Odds'].astype(float)
clay_today['Fav_Odds']=clay_today['Fav_Odds'].astype(float)
clay_today['Fav_Elo']=clay_today.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']==x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
clay_today['Dog_Elo']=clay_today.apply(lambda x: x['Elo_Fav_Elo'] if x['Fav']!=x['Elo_Fav'] else x['Elo_Dog_Elo'],axis=1)
clay_today['rank_diff']=clay_today['dog_rank']-clay_today['fav_rank']
clay_today['elo_diff']=clay_today['Fav_Elo']-clay_today['Dog_Elo']
clay_filtered=clay_today
clay_filtered=clay_filtered[(clay_filtered['Dog_Odds']).ge(2)&(clay_filtered['Dog_Odds']).le(2.2)]
clay_input=clay_filtered[['fav_percent','dog_percent','rank_diff','elo_diff']]



if len(clay_input)>0:

    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    import numpy as np

    # Initialize an empty list to store probabilities from each iteration
    all_probabilities = pd.DataFrame()
    counter=0
    for q in range(1, 1000):
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            merge[['fav_percent','dog_percent','rank_diff','elo_diff']],
            merge['Win'],
            test_size=0.3,
        )

        # Create a logistic regression model and fit it to the training data
        logreg = LogisticRegression(max_iter=1000000000000000000000)
        logreg.fit(X_train, y_train)

        # Evaluate the model on the testing data
        score = logreg.score(X_test, y_test)

        if score > 0.6:
            counter=counter+1
            clay_input = clay_filtered[['fav_percent','dog_percent','rank_diff','elo_diff']].copy()

            # Predict the probability of each sample belonging to class 1
            prob = logreg.predict_proba(clay_input)[:, 1]

            # Add 'Probability' column to the input DataFrame
            clay_input['Probability'] = prob

            xo = clay_input.merge(clay_filtered, left_index=True, right_index=True, suffixes=['','_y'])[['Dog','Dog_Odds','Probability','Time','Fav','Fav_Odds']]
            #print(xo)
            #print(all_probabilities)
            all_probabilities=pd.concat([all_probabilities,xo])

    avg_probs = all_probabilities.groupby(['Time','Dog_Odds','Dog','Fav','Fav_Odds'])['Probability'].mean().reset_index()
    print(counter)
    avg_probs['Prediction']=avg_probs.apply(lambda x: True if x['Probability']>=0.5 else False,axis=1 )
    avg_probs['Date']=Date
    avg_probs['Surface']='Clay'
    avg_probs['Result']=None
    if push_to_sql:
        avg_probs[['Date','Dog_Odds','Fav_Odds','Dog','Fav','Prediction','Probability','Surface']].to_sql('Predictions_Today',con=devengine,index=False,if_exists='append')

    today=avg_probs[['Time','Dog_Odds','Dog','Fav','Fav_Odds','Prediction','Probability']]
else:
    today=pd.DataFrame()



214


In [185]:
todays=pd.concat([today_hard,today]).sort_values(by='Time')
todays#[todays['Prediction']==True]

,Time,Dog_Odds,Dog,Fav,Fav_Odds,Prediction,Probability
0,20:30,2.15,Borna Coric,Tomas Martin Etcheverry,1.73,False,0.443046
1,21:00,2.06,Ekaterina Alexandrova,Beatriz Haddad Maia,1.79,False,0.422607
2,22:30,2.08,Bernarda Pera,Elisabetta Cocciaretto,1.78,False,0.489393
3,22:30,2.11,Francisco Cerundolo,Taylor Fritz,1.75,True,0.512740
4,22:30,2.14,Lesia Tsurenko,Bianca Andreescu,1.73,True,0.501316


In [186]:
todays

,Time,Dog_Odds,Dog,Fav,Fav_Odds,Prediction,Probability
0,20:30,2.15,Borna Coric,Tomas Martin Etcheverry,1.73,False,0.443046
1,21:00,2.06,Ekaterina Alexandrova,Beatriz Haddad Maia,1.79,False,0.422607
2,22:30,2.08,Bernarda Pera,Elisabetta Cocciaretto,1.78,False,0.489393
3,22:30,2.11,Francisco Cerundolo,Taylor Fritz,1.75,True,0.512740
4,22:30,2.14,Lesia Tsurenko,Bianca Andreescu,1.73,True,0.501316
